In [6]:
! pip install google-generativeai

  Obtaining dependency information for google-generativeai from https://files.pythonhosted.org/packages/9b/b0/6c6af327a8a6ef3be6fe79be1d6f1e2914d6c363aa6b081b93396f4460a7/google_generativeai-0.8.4-py3-none-any.whl.metadata
  Obtaining dependency information for google-ai-generativelanguage==0.6.15 from https://files.pythonhosted.org/packages/7c/a3/67b8a6ff5001a1d8864922f2d6488dc2a14367ceb651bc3f09a947f2f306/google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core from https://files.pythonhosted.org/packages/46/95/f472d85adab6e538da2025dfca9e976a0d125cc0af2301f190e77b76e51c/google_api_core-2.24.2-py3-none-any.whl.metadata
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/b4/44/ae1528a6ca296d89704c8febb72b3e263c28b4e50ab29b9202df7a0f273d/google_api_python_client-2.166.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0dev,>=1.22.3 from

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.29.4 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.


In [1]:
import json
import http.client
conn = http.client.HTTPSConnection("booking-com15.p.rapidapi.com")
headers = {
        'x-rapidapi-key': "cd3649e95amsh19e77450d847972p129963jsn55e782c26fc6",
        'x-rapidapi-host': "booking-com15.p.rapidapi.com"
    }

In [2]:
input_data = {
    "Amsterdam": {
        "Arrival Date": "2025-06-01",
        "Departure Date": "2025-06-03",
        "Airport": "AMS"
    },
    "The Hague": {
        "Arrival Date": "2025-06-03",
        "Departure Date": "2025-06-05",
        "Airport": "RTM"                      
    },
    "Maastricht": {
        "Arrival Date": "2025-06-05",
        "Departure Date": "2025-06-08",
        "Airport": "MST"
    },
    "Eindhoven": {
        "Arrival Date": "2025-06-08",
        "Departure Date": "2025-06-10",
        "Airport": "EIN"
    }
}

In [3]:
import http.client
import urllib.parse

def JSONconverter(input):
    cities = list(input.keys())
    arrivals = []
    departures = []
    airports = []

    encoded_cities = []  # To store URL-encoded city names

    for keys, values in input.items():
        encoded_city = urllib.parse.quote(keys)  # URL-encode the city name
        encoded_cities.append(encoded_city)
        
        arrivals.append(values["Arrival Date"])
        departures.append(values["Departure Date"])
        airports.append(values["Airport"])

    return cities,encoded_cities, arrivals, departures, airports


In [4]:
def structure_flight_data(flight_dict):
    structured_data = []

    for (from_city, to_city, date), details in flight_dict.items():
        if isinstance(details, str):  # case: no flights
            structured_data.append({
                "from": from_city,
                "to": to_city,
                "date": date,
                "flights": None,
                "note": details
            })
        else:
            flight_options = []
            for category, (price, dep_time, arr_time) in details.items():
                flight_options.append({
                    "type": category,
                    "price_usd": price,
                    "departure_time": dep_time,
                    "arrival_time": arr_time
                })

            structured_data.append({
                "from": from_city,
                "to": to_city,
                "date": date,
                "flights": flight_options,
                "note": None
            })

    return structured_data


In [5]:
import http.client
import json
def FlightRetriever(input,numadults):
    FullyFinalFlightHash={}
    for i in range(len(input)-1):
            real_cities,cities,_,departures,airports = JSONconverter(input)
            start = airports[i]
            end = airports[i+1]
            start_city = cities[i]
            end_city = cities[i+1] 
            real_start_city = real_cities[i]
            real_end_city = real_cities[i+1]
            departuredate = departures[i]
            TempFlightHash = {}
            FinalFlightHash={}
            conn = http.client.HTTPSConnection("booking-com15.p.rapidapi.com") 
            url = f"/api/v1/flights/searchFlights?fromId={start}.AIRPORT&toId={end}.AIRPORT&pageNo=1&adults={numadults}&children=0%2C17&sort=BEST&cabinClass=ECONOMY&currency_code=USD&departDate={departuredate}"
            try:
                conn.request("GET", url, headers=headers)
                res = conn.getresponse()
                data = res.read()
                data_json = json.loads(data.decode("utf-8"))
                # flight_data[(start, end)] = data_json
                # print(f"Fetched flights from {start} to {end}")
            except Exception as e:
                pass
                # print(f"Error fetching data from {start} to {end}: {e}")
            finally:
                conn.close()  # Always close the connection
            #Edge Case when there are no flights
            if 'error' in list(data_json['data'].keys()):
                FullyFinalFlightHash[(real_start_city,real_end_city,departuredate)] = "No flights for these locations.Check alternate way of travel!"
                continue
            for flights in data_json['data']['flightDeals']:
                TempFlightHash[flights['key']] = (flights['priceRounded']['units'],flights['offerToken'])
            for sorts,tokens in TempFlightHash.items():
                token = tokens[1]
                for flights in data_json['data']['flightOffers']:
                    if token == flights['token']:
                        FinalFlightHash[sorts]=(tokens[0],flights['segments'][0]['departureTime'],flights['segments'][0]['arrivalTime'])
            FullyFinalFlightHash[(real_start_city,real_end_city,departuredate)] = FinalFlightHash
    return structure_flight_data(FullyFinalFlightHash)


In [6]:
dataa = FlightRetriever(input_data,3)

In [7]:
dataa

[{'from': 'Amsterdam',
  'to': 'The Hague',
  'date': '2025-06-03',
  'flights': [{'type': 'CHEAPEST',
    'price_usd': 810,
    'departure_time': '2025-06-03T18:40:00',
    'arrival_time': '2025-06-04T12:10:00'},
   {'type': 'FASTEST',
    'price_usd': 1226,
    'departure_time': '2025-06-03T16:20:00',
    'arrival_time': '2025-06-03T18:55:00'},
   {'type': 'BEST',
    'price_usd': 1226,
    'departure_time': '2025-06-03T16:20:00',
    'arrival_time': '2025-06-03T18:55:00'}],
  'note': None},
 {'from': 'The Hague',
  'to': 'Maastricht',
  'date': '2025-06-05',
  'flights': None,
  'note': 'No flights for these locations.Check alternate way of travel!'},
 {'from': 'Maastricht',
  'to': 'Eindhoven',
  'date': '2025-06-08',
  'flights': None,
  'note': 'No flights for these locations.Check alternate way of travel!'}]